Designed by: [Link](https://github.com/MakarenaLabs/Common-PL-Devices-on-PYNQ/blob/main/SPI/spi_test_pmodB.ipynb). Also uses Sudhanshu's code [here](https://github.com/dakshasat/dev-electronics/blob/stc_prototyping/STC/Sudhanshu/notebooks/test_2det_commanding.ipynb). Comments and changes by Ashwajit

Create Date: 11/10/2024
Component Name: Pynq SPI Control
Description: Contains the code to send commands and receive data via SPI to and from the Pynq board

Dependencies: NA

Revision:

Additional Comments:


In [1]:
from pynq import Overlay

## Overlay

In [2]:
ol = Overlay("pynqz2_spi_pmodB.bit") # Ensure bit and hwh files are present

In [3]:
mkl_spi = ol.axi_quad_spi_0

## SPI Control

In [4]:
XSP_DGIER_OFFSET = 0x1C
XSP_IISR_OFFSET = 0x20
XSP_IIER_OFFSET = 0x28
XSP_SRR_OFFSET = 0x40
XSP_CR_OFFSET = 0x60 # Control register
XSP_SR_OFFSET = 0x64 # Status register
XSP_DTR_OFFSET = 0x68 # Data transfer register
XSP_DRR_OFFSET = 0x6C # Data receive register
XSP_SSR_OFFSET = 0x70 # Register to select slave, one hot encoding
XSP_TFO_OFFSET = 0x74
XSP_RFO_OFFSET = 0x78
XSP_REGISTERS = [0x40, 0x60, 0x64, 0x68, 0x6c, 0x70, 0x74, 0x78, 0x1c, 0x20, 0x28]

XSP_SRR_RESET_MASK = 0x0A
XSP_SR_TX_EMPTY_MASK = 0x04
XSP_SR_TX_FULL_MASK = 0x08
XSP_CR_TRANS_INHIBIT_MASK = 0x100
XSP_CR_LOOPBACK_MASK = 0x01
XSP_CR_ENABLE_MASK = 0x02
XSP_CR_MASTER_MODE_MASK = 0x04 # 0x04 for 1
XSP_CR_CLK_POLARITY_MASK = 0x08 # CPOL = 0, 0x08 for 1
XSP_CR_CLK_PHASE_MASK = 0x10 # CPHA = 0, change to 0x10 for 1
XSP_CR_TXFIFO_RESET_MASK = 0x20
XSP_CR_RXFIFO_RESET_MASK = 0x40
XSP_CR_MANUAL_SS_MASK = 0x80

SLAVE_NO_SELECTION = 0xFFFFFFFF

def cnfg(AxiQspi, clk_phase=0, clk_pol=1):
    print("Configure device")
    # Reset the SPI device
    AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
    # Enable the transmit empty interrupt, which we use to determine progress on the transmission. 
    AxiQspi.write(XSP_IIER_OFFSET, XSP_SR_TX_EMPTY_MASK)
    # Disable the global IPIF interrupt
    AxiQspi.write(XSP_DGIER_OFFSET, 0)
    # Deselect the slave on the SPI bus
    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    # Disable the transmitter, enable Manual Slave Select Assertion, put SPI controller into master mode, and enable it
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK | XSP_CR_TXFIFO_RESET_MASK | XSP_CR_RXFIFO_RESET_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~(XSP_CR_CLK_PHASE_MASK | XSP_CR_CLK_POLARITY_MASK) 
    if clk_phase == 1:
        ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
    if clk_pol == 1:
        ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    return 0

def xfer(packet, AxiQspi):
    print("TransferData")
    for data in packet:
        AxiQspi.write(XSP_DTR_OFFSET, data) # Data transfer register
        AxiQspi.write(XSP_SSR_OFFSET, 0xFFFFFFFE) # Selects slave
        ControlReg = AxiQspi.read(XSP_CR_OFFSET) 
        ControlReg = ControlReg & ~XSP_CR_TRANS_INHIBIT_MASK # Allows master transactions
        AxiQspi.write(XSP_CR_OFFSET, ControlReg) 

        StatusReg = AxiQspi.read(XSP_SR_OFFSET) 
        while (StatusReg & XSP_SR_TX_EMPTY_MASK) == 0:
            StatusReg = AxiQspi.read(XSP_SR_OFFSET) # Until Tx_empty becomes high, continues to loop

        print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(AxiQspi.read(XSP_RFO_OFFSET))) # I don't exactly understand what this does, it's just the number of spots of receive FIFO that are used
        # May be useful to check if you should now only receive
        ControlReg = AxiQspi.read(XSP_CR_OFFSET) 
        ControlReg = ControlReg | XSP_CR_TRANS_INHIBIT_MASK # Disable master transaction
        AxiQspi.write(XSP_CR_OFFSET, ControlReg) 
        print('Sent ', data)

    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION) #
    return 

def get_response(AxiQspi):
    print("ReadResponse")
    resp = list()
    RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01 
    while RxFifoStatus == 0: # 0 while Rx is still full
        temp = AxiQspi.read(XSP_RFO_OFFSET) # Packet number essentially
        print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(temp))
        temp = AxiQspi.read(XSP_DRR_OFFSET) # Data
        print('XSP_DRR_OFFSET  : 0x{0:08x}'.format(temp))    
        resp.append(temp)
        RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
    return resp




## CZT Specific Code

In [ ]:
_command_map = {"READ_PART_BASE": 0xE0,
                    "READ_SERIAL_LSB": 0x9D,
                    "READ_SERIAL_MSB": 0x9E,
                    "READ_FIRMWARE_VERSION": 0x86,
                    "READ_MODULE_VERSION": 0xA3,
                    "READ_STATUS": 0x96,
                    "READ_TEMPERATURE": 0x9A,
                    "BREAK": 0x02,
                    "EVENT_ON": 0x85,
                    "EVENT_OFF": 0x05,
                    "FIFO_CLEAR": 0x8C,
                    "SET_THRESHOLD": 0x21,
                    "GET_THRESHOLD": 0xA1,
                    "SET_GPIO": 0x1F,
                    "GET_GPIO": 0x9F,
                    "SET_CLOCK": 0x20,
                    "GET_CLOCK": 0xA0,
                    "RESTORE_SETUP": 0x81,
                    "UPDATE_SETUP": 0x01,
                    "SET_PEAKING_TIME": 0x32,
                    "GET_PEAKING_TIME": 0xB2,
                    "RUN_SELF_TEST": 0x34,
                    "GET_SELF_TEST_RESULTS": 0xB4,
                    "SET_CHANNEL": 0x07,
                    "GET_CHANNEL": 0x87,
                    "CHANNEL_CONTROL": 0x0B,
                    "CHANNEL_STATUS": 0x8B,
                    "SET_ADDR_POINTER": 0x03,
                    "GET_ADDR_POINTER": 0x83,
                    "READ_RAM": 0x84,
                    "WRITE_RAM": 0x04,
                    "EEPROM_CHECKSUM": 0xCB,
                    "HOLD_ON": 0x10,
                    "HOLD_OFF": 0x90,
                    "SET_EMULATOR": 0x48,
                    "GET_EMULATOR": 0xC8,
                    "WRITE_PROTECT_OFF": 0x4A,
                    "WRITE_PROTECT_ON": 0xCA
                   }

_command_valids = [0xE0,0xE1,0xE2,0xE3,0xE4,0xE5,0xE6,0xE7,0xE8,0xE9,0x9D,0x9E,0x86,0xA3,0x96,0x9A,0x02,0x85,0x05,0x8C,0x21,0xA1,0x1F,0x9F,0x20,0xA0,0x81,0x01,0x32,0xB2,0x34,0xB4,0x07,0x87,0x0B,0x8B,0x03,0x83,0x84,0x04,0xCB,0x10,0x90,0x48,0xC8,0x4A,0xCA]
_command_has_reply = [0xE0,0x9D,0x9E,0x86,0xA3,0x96,0x9A,0xA1,0x9F,0xA0,0xB2,0xB4,0x87,0x8B,0xC8,0x83,0x84,0xCB]
_command_has_data = [0x21,0x1F,0x20,0x32,0x07,0x0B,0x48,0x03,0x04]


In [5]:
cnfg(mkl_spi)

Configure device


0

## Testing

In [7]:
xfer([0x85000000], mkl_spi)

TransferData
XSP_RFO_OFFSET  : 0x00000000
ReadResponse
XSP_RFO_OFFSET  : 0x00000000
XSP_DRR_OFFSET  : 0x00000000
